# Simulating with the QUBE FF

Now that we have prepared a QUBE input pdb and force field xml file we can simulate the system using OpenMM. However, there are some things we have to do when using OpenMM and QUBE to make sure it runs correctly. 

Firstly, depending on the size of the system you may get a recursion limit exceeded error when you try to apply the force field to the system, this can be overcome by manually setting a limit higher than the number of atoms in the protein. To do this just import the setrecursionlimit function from sys.

Here we will set up a simulation of the capped LEU residue which should not produce this error and we will use a function from QUBEKit to apply the opls geometric combination rules.

In [ ]:
# standard OpenMM imports
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit

# load the initial coords into the system using the special QUBE_pro.pdb file
pdb = app.PDBFile('QUBE_pro.pdb')

# now load in the QUBE_pro.xml force field file and the selected water model 
forcefield = app.ForceField('QUBE_pro.xml', 'QUBE_tip4p-d.xml')
# set the intial positions from the pdb
modeller = app.Modeller(pdb.topology, pdb.positions)

# now we want to solvate the system, here we use the tip4pew model when placing waters
# this makes sure the internal geometry of each water molecule is correct
# we also use neutralize False as we do not have an ion model for the QUBE FF
modeller.addSolvent(forcefield, model='tip4pew', padding=1 * unit.nanometers, neutralize=False)

# now we can write out a solvated file to make sure it looks as we expect
app.PDBFile.writeFile(modeller.topology, modeller.positions, open('output.pdb', 'w+'))

Now lets view the file to make sure it looks right using nglview.

In [ ]:
import nglview as nv
import pytraj as pt
from IPython.display import display

traj = pt.load('output.pdb')
view = nv.show_pytraj(traj)
view.add_representation('line', selection='HOH')
view

Next we will create the OpenMM system objects and add the opls combination rule.

In [ ]:
# now we create the system make sure to use rigidWater for certain models.
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, constraints=app.HBonds, nonbondedCutoff=1 * unit.nanometer)

# now we can import the opls helper from QUBEKit
from QUBEKit.proteinTools import apply_opls_combo

system = apply_opls_combo(system, switching_distance=0.95 * unit.nanometer)

# set control parameters
temperature = 298.15 * unit.kelvin
integrator = mm.LangevinIntegrator(temperature, 5 / unit.picoseconds, 0.001 * unit.picoseconds)

# add preasure to the system
system.addForce(mm.MonteCarloBarostat(1 * unit.bar, 298.15 * unit.kelvin))

# create the simulation context
simulation = app.Simulation(modeller.topology, system, integrator)

# set the positions to the solvated system
simulation.context.setPositions(modeller.positions)


The system is now ready to be run.

In [ ]:
# now minimize the system
print('Minimizing...')
simulation.minimizeEnergy(maxIterations=100)

# set temperatures
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
print('Equilibrating...')
simulation.step(100)

# now run the simulation
simulation.reporters.append(app.PDBReporter('run.pdb', 1000))
simulation.reporters.append(app.StateDataReporter('run.txt', 1000, step=True, potentialEnergy=True, temperature=True))
print('Production run...')
simulation.step(100000)
print('Done!')

Once the simulation is complete we can load the results using nglview and pytraj once again.

In [ ]:
traj = pt.load('run.pdb')
view = nv.show_pytraj(traj)
view.add_representation('line', selection='HOH')
view

# Convert

If we look at the pdb file produced by the simulation run we can see that the atom names and residue names are still in the QUBE style which can make post anylsis hard. QUBEKit comes with a function that can back transform the pdb file into a more sutible format.

In [ ]:
# lets check the OpenMM output
pdb = open('run.pdb', 'r').readlines()
pdb

In [ ]:
from QUBEKit.proteinTools import pdb_reformat
# pdb reformat takes a reference pdb and a conversion target
reference = 'capped.pdb'  # orginal input file
target = 'run.pdb'        # the OpenMM output file
pdb_reformat(reference, target)

In [ ]:
# Lets cehck the new output file QUBE_traj.pdb
qube = open('QUBE_traj.pdb', 'r').readlines()
qube


In [ ]:
traj = pt.load('QUBE_traj.pdb')
view = nv.show_pytraj(traj)
view.add_representation('licorice', selection='QUP')
view.add_representation('line', selection='HOH')

view